In [26]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [27]:
url = "https://guide.wisc.edu/undergraduate/business/risk-insurance/business-actuarial-science-bba/#requirementstext"
page = requests.get(url)

In [28]:
text = BeautifulSoup(page.text, "html.parser")

In [29]:
# Get the headings for the previous tables

# First, we find all the tables and only keep the tables containing course info
tables = text.findAll("table", {"class": "sc_courselist" })

#required core part
table_names = []
#for table in tables[4]:
    #table_names.append(table.find("span", {"class": "odd areasubheader undefined"}))

In [30]:
def get_ele(x):
    for ele in x:
        return ele.get_text()
    
def replace_that(table):
    a = []
    for row in table:
        r = []
        credits = row.findAll("td", {"class": "hourscol" })
        credits = get_ele(credits)
        r.append(credits)
        
        catg = row.findAll("h2",{"name":"requirementstext"})
        catg = get_ele(catg)
        r.append(catg)
        
        classtags= row.findAll("td", {"class": "codecol"})
        if len(classtags) > 0:
            classtags = get_ele(classtags).replace(u'\xa0',' ')
            classtags = classtags.replace(u'\u200b',' ')
        r.append(classtags)
        
        a.append(r)
    return(a)


In [31]:
#Now, we aim to clean up tb
def clean_up(tb, cat = "None"):
    categories = []
    for row in tb:
        ## Categories
        if row[0] != None and len(row[0]) > 0:
            ## Categories
            if row[0]=="1":
                row[0] = round(int(row[0])/3)
            elif row[0]=="3":
                row[0] = round(int(row[0])/3)
            category = ({"Name":cat},
                        row[0],[row[2]])
            categories.append(category)
        elif len(row[2]) > 0:
            category[-1].append(row[2])
    return categories

#categories = categories[:-1]
#categories.pop(7)


[({'Name': 'Actuarial Requirement'},
  1,
  ['MATH/ STAT  431', 'or STAT/ MATH  309', 'or STAT 311']),
 ({'Name': 'Actuarial Requirement'}, 1, ['STAT/ MATH  310', 'or STAT 312']),
 ({'Name': 'Actuarial Requirement'}, 0, ['ACT SCI 300']),
 ({'Name': 'Actuarial Requirement'}, 0, ['ACT SCI 301']),
 ({'Name': 'Actuarial Requirement'}, 1, ['ACT SCI 303']),
 ({'Name': 'Actuarial Requirement'}, 1, ['ACT SCI 650']),
 ({'Name': 'Actuarial Requirement'}, 1, ['ACT SCI 652']),
 ({'Name': 'Actuarial Requirement'}, 1, ['ACT SCI 651', 'or ACT SCI 653']),
 ({'Name': 'Actuarial Requirement'},
  1,
  ['ACT SCI 654', 'or ACT SCI 655', 'or GEN BUS 656']),
 ({'Name': 'Actuarial Requirement'}, '23', [[]])]

In [35]:
## INFO FROM Major Requirement
major_tbody = tables[1].findAll("tbody")
major_tr=major_tbody[0].findAll("tr")
major_req = replace_that(major_tr)
major = clean_up(major_req, "Actuarial Science Major Requirements")

## INFO FROM ELECTIVE Requirement
elective_tbody = tables[2].findAll("tbody")
elective_tr=elective_tbody[0].findAll("tr")
elective_req = replace_that(elective_tr)
elective = clean_up(elective_req, "Recommended Electives")